

```
Mora u jedan cell block zato sto je klasa
```



In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [ ]:
df[:3755].to_csv("data/modified.csv", index=False)

df = pd.read_csv('data/modified.csv')
name = df['name']
tag =  df['tag']
points = df.drop(columns=['name', 'tag'])
#points = np.array(points.values)
points = points.values.tolist()

# dataPoints = []
	# for i in range(len(points)):
# 	person = []
# 	for j in range(0, len(points[i]), 2):
# 		sample = [float(points[i][j]), float(points[i][j+1])]
# 		person.append(sample)
# 	dataPoints.append(person)

file = open("data/points.json", "w")
json.dump(points, file)

In [ ]:
class ShapeModel:
	def __init__(self):
		self.p = np.mat([]) # Parameter vector
		self.V = np.mat([])	# Shape basis
		self.e = np.mat([]) # Parameter variance
		self.C = np.mat([]) # Connectivity


	"""
	npts
	Number of points in shape model
	input:
		-
	output:
		-number of points
	"""
	def npts(self):
		return self.V[:] / 2 


	"""
	calc_params
	input:
		-pts: points to compute parameters from (vector<Point2f>)
		-weight: weight of each point (Mat)
		-c_factor(default=3.0): clamping factor (float)
	output:
		-
	"""
	def calc_params(self, pts, weight, c_factor=3.0):
		n = len(pts)
		assert len(self.V) == 2*n
		s = np.mat(pts).reshape((1, 2*n)) #point set to vector format

		if(len(weight) == 0): # FIX
			p = self.V.T * s #simple projections

		else: #scaled projection
			if(len(weight) != n):
				print("Invalid weighting matrix")
				return

			K = self.V.shape[1]
			H = np.mat(np.zeros((K, K)))
			g = np.zeros((K, 1))

			for i in range(n):
				v = np.mat(cv2.rectangle(0, 2*i, K, 2)) # FIX
				w = np.array(weight[i], dtype="float32")
				H+=w*v.T*v
				g+=w*v.T*np.mat((pts[i])) # FIX THIS
				
			cv2.solve(H, g, self.p, method=cv2.DECOMP_SVD)	
		clamp(c_factor)
	

	"""
	calc_shape
	input:
		-
	output:
		-shape described by parameters
	"""
	def calc_shape(self):
		s = np.mat(self.V*self.p)
		n = len(s) / 2
		pts = []
		for i in range(int(n)):
			p = [s[2*i], s[2*i+1]]
			pts.append(p)
		return pts
	

	"""
	center_shape
	input:
		-pts: shape to center (Mat)
	output:
		-centered shape
	"""
	def center_shape(self, pts):
			n = int(len(pts) / 2)
			mx = 0.
			my = 0.

			for i in range(n):
				mx += pts[2*i] # Sum ?
				my += pts[2*i+1] # Sum ?
			p = np.mat(np.zeros((2*n, 1)))
			mx /= n
			my /= n
			for i in range(n):
				p[2*i] = pts[2*i] - mx #sum ?
				p[2*i+1] = pts[2*i+1] - my #sum ?

			return p


	"""
	Set identity
	input:
		-
	output:
		-
	"""
	def set_identity_params(self):
		self.p[:] = 0
		self.p[0] = 1.0


	"""
	pts2mat
	input:
		-points: points to vectorise (vector<Point2F>)
	output:
		- matrix
	"""
	def pts2mat(self, points):
		print("Converting to Mat..")
		N = len(points)
		assert N > 0

		n = len(points[0])
		for i in range(1, N):
			assert len(points[i]) == n
		X = np.mat(np.zeros((n, N)), dtype='float64')

		for i in range(N):
			x = X[:, i].reshape((1, -1))
			y = np.mat(points[i])
			np.copyto(x, y)
		print("Success!")
		return X


	"""
	procrustes
	input:
		-X: shapes to align (mat)
		-itol(100): maximum number of iterations (int)
		-ftol(1e-6) convergence tolerance (float)
	output:
		-matrix with procrusted aligned shapes/column
	"""
	def procrustes(self, X, itol=100, ftol=1e-8):
		colormap = plt.cm.jet
		colorst = [colormap(i) for i in np.linspace(0, 0.9, 76)]
		print(">Starting procrustes analysis...")

		n = int(np.size(X, 0) / 2)
		N = int(np.size(X, 1))

		fig, axs = plt.subplots(1, 2)
		axs[0].set_title("Before procrustes")
		axs[1].set_title("After procrustes")

		#remove centre of mass
		P = X[:]
		for i in range(0, N, 100):

			p = P[:, i]
			mx, my = 0, 0
			for j in range(n):
				axs[0].scatter(np.float32(p[2*j, 0]), 480 - np.float32(p[2*j+1, 0]), c=np.atleast_2d(colorst[j])) # Ovaj deo iscrtava
				axs[0].axis('equal')
				axs[0].axis('off')
				mx += p[2*j]
				my += p[2*j+1]
			mx = mx / N
			my = my / N
			for j in range(n):
				p[2*j]  -= mx
				p[2*j+1]-= my

		#optimise scale and rotation
		C_old = np.mat(np.zeros((152, 1)))
		for iters in range(itol):

			C = np.mat(P*np.ones((N, 1))/N)
			cv2.normalize(C, C)

			if(iters>0):
				if(cv2.norm(C, C_old) < ftol):
					break
				C_old = C[:]

				print(">>Starting rotation, alignment and translation...")
				for i in range(0, N, 100):

					R = np.mat(self.rot_scale_align(P[:, i], C))
					for j in range(n):

						print(f"Iteration {iters}, face {i}, landmark {j}")			
						x = P[2*j, i]
						y = P[2*j+1, i]
						P[2*j, i] = R[0, 0] * x + R[0, 1] * y
						P[2*j+1, i] = R[1, 0] * x + R[1, 1] * y
						axs[1].scatter(P[2*j, i], 480 - P[2*j+1, i], c=np.atleast_2d(colorst[j])) # Ovaj deo iscrtava
				axs[1].axis('equal')
				axs[1].axis('off')
				plt.show()
				print(">>Success !")

		print(">Finished procrustes!")
		return P


	"""
	rot_scale_align
	input:
		-src: Source point (mat)
		-dst: Destination point (mat)
	output:
		-scaled rotation matrix
	"""
	def rot_scale_align(self, src, dst):
		#construct linear system
		n = int(len(src) / 2)
		a, b, d = 0, 0, 0
		for i in range(n):
			d += src[2*i] * src[2*i  ] + src[2*i+1] * src[2*i+1] # : ?
			a += src[2*i] * dst[2*i  ] + src[2*i+1] * dst[2*i+1] # : ?
			b += src[2*i] * dst[2*i+1] - src[2*i+1] * dst[2*i  ] # : ?

		#solved linear system
		a = float(a / d)
		b = float(b / d)

		return np.mat([[a, -b], [b, a]])


	"""
	calc_rigid_basis
	input:
		-X: procrustes aligned shapes/column
	output:
		-rigid basis
	"""
	def calc_rigid_basis(self, X):
		print(">Starting to calculate rigid basis...")


		#Compute mean shape
		n = int(np.size(X, 0) / 2)
		N = int(np.size(X, 1))
		mean = X * np.ones((N, 1)) / N

		#construct basis for similarity transform
		R = np.mat(np.zeros((2*n, 4)))
		for i in range(n):
			R[2*i  , 0] = mean[2*i]
			R[2*i+1, 0] = mean[2*i+1]

			R[2*i  , 1] = -mean[2*i+1]
			R[2*i+1, 1] = mean[2*i]

			R[2*i  , 2] = 1.
			R[2*i+1, 2] = 0.
			R[2*i  , 3] = 0.
			R[2*i+1, 3] = 1. 

		
		#Gram-Schmidt orthonormalization
		for i in range(4):
			r = np.mat(R[:, i])
			for j in range(i):
				b = np.mat(R[:, j])
				r -= b * (b.transpose() * r)
			cv2.normalize(r, r)
		print(">Success!")
		return R


	"""
	clamp
	input:
		-c(3.0): clamping factor(or standard dev) (float)
	output:
		-
	"""
	def clamp(self, c = 3.0):
		scale = self.p[0]
		for i in range(np.size(self.e, 1)):
			if(float(self.e[:, i]) < 0):
				continue
			v = float(c * np.sqrt(self.e[:, i]))
			if(np.any(np.abs(self.p[:, i] / scale) > v)):
				if(float(self.p[:, i]) > 0):
					p[:, i] = v * scale
				else:
					p[:, i] = -v* scale


	"""
	train
	input:
		-points: N-example shapes (vector<vector<Point2f>>)
		-con: point connectivity (vector<Vec2i>)
		-frac(0.95): fraction of variation to retain (float)
		-kmax(10): maximum number of modes to retain (int)
	output:
		-
	"""
	def train(self, points, con, frac=0.95, kmax=10):
		X = self.pts2mat(points)
		n = int(np.size(X, 0) / 2)
		N = int(np.size(X, 1))

		Y = self.procrustes(X)

		R = self.calc_rigid_basis(Y)

		P = R.T * Y 
		dY = Y - R*P
		w, u, vt = cv2.SVDecomp(dY*dY.T)
		
		m = min(min(kmax, N-1), n-1)
		vsum = 0.
		for i in range(m):
			vsum += w[i]
		v, k = 0., 0.
		for k in range(m):
			v+= w[k]
			if(v / vsum >= frac):
				k+=1
				break
		if(k > m):
			k = m
		D = u[0:k, 0:2*n]

		self.V = np.mat(np.empty((2*n, 4+k)))
		Vr = self.V[0:4, 0:2*n]
		#np.copyto(Vr, R) # FIX
		#Vd = self.V[4:k, 0, 2*n] # FIX
		#np.copyto(Vd, D) # FIX

		Q = self.V.T * X
		for i in range(N):
			v = Q[0, i]
			q = Q[:, i]
			q = q / v

		self.e = np.mat(np.empty((4+k, 1)))

		Q = cv2.pow(Q, 2, Q)
		for i in range(4+k):
			if(i < 4):
				self.e[i] = -1
			else:
				self.e[i] = np.dot(Q[i], np.ones((N, 1))) / (N-1)

		if(len(con) > 0):
			m = len(con)
			self.C = np.mat(np.empty((m, 2)))
			for i in range(m):
				self.C[i, 0] = con[i][0]
				self.C[i, 1] = con[i][1]
		else:
			self.C = np.mat(np.empty((n, 2)))
			for i in range(n-1):
				self.C[i, 0] = i
				self.C[i, 1] = i+1
			self.C[n-1, 0] = n-1
			self.C[n-1, 1] = 0
		print("Training complete")
	
	"""
	write
	input:
		-fs: file storage object to write to
	output:
		-
	"""
	def write(self, fs=cv2.FileStorage()):
		pass


	"""
	read
	input:
		-
	output:
		-
	"""
	def read(self):
		pass


In [ ]:
file = open("data/points.json", "r")
points = json.load(file)

model = ShapeModel()
model.train(points, [])